In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4397/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_4397/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_7_8,0.871302,0.652926,0.775313,0.769352,0.305039,2.320886,0.803694,0.284684,1.071908,0.552303,0.734301,0.561001,68.374632,140.129412,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_8_2,0.877100,0.650697,0.780600,0.869380,0.291296,2.335789,0.784781,0.138770,1.057612,0.539719,0.746271,0.548219,68.466830,140.221610,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_6_3,0.840048,0.648638,0.707866,0.891848,0.379117,2.349556,1.044948,0.411200,1.318536,0.615725,0.669777,0.625422,67.939822,139.694602,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_7_3,0.862922,0.648111,0.765775,0.791559,0.324900,2.353085,0.837808,0.257274,1.171949,0.570000,0.717001,0.578977,68.248474,140.003254,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_4_7,0.879906,0.647668,0.789912,0.833269,0.284646,2.356044,0.751473,0.295383,1.249341,0.533522,0.752064,0.541925,68.513019,140.267799,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,model_2_49_6,0.895794,-73.429999,0.695321,0.837879,0.246988,497.713746,1.089819,0.251641,0.906996,0.496979,0.784866,0.504806,68.796833,140.551613,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1310,model_2_46_6,0.883530,-79.529956,0.708273,0.827201,0.276056,538.504184,1.043491,0.505431,0.881926,0.525410,0.759546,0.533685,68.574304,140.329084,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1311,model_2_49_9,0.895273,-89.271168,0.694729,0.823391,0.248224,603.643719,1.091935,0.274128,0.918603,0.498221,0.783789,0.506067,68.786849,140.541629,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1312,model_2_46_3,0.882773,-89.471102,0.706572,0.833616,0.277851,604.980675,1.049574,0.486666,0.904191,0.527115,0.757983,0.535417,68.561344,140.316124,"Hidden Size=[8], regularizer=0.02, learning_ra..."
